The following command adds the pyspark to sys.path at runtime. If the pyspark is not on the system path by default. It also prints the path of the spark.

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark

Create a Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Hyperparameter") \
    .master('local[3]') \
    .getOrCreate()

Read the dataset into a dataframe.

In [ ]:
customer_churn = spark.read.csv('customer_churn_exercise.csv', inferSchema=True, header=True, mode='DROPMALFORMED')

About the Dataset
This is the data of the marketing agency which has altogether 8 features and 1 target variable.

Name: Name of the company whom the customer is tagged to
Age: Age of the Customer
Total_Purchase: Total Ads Purchased
Account_Manager: Binary 0=No manager, 1= Account manager assigned
Years: Total Years of customers using the company service
Num_sites: Total number of websites that are using this service.
Onboard_date: Onboarding date of the latest contacted person.
Location: Head Quarter address of the client
Company: Name of Client’s Company

In [ ]:
customer_churn.show(1, truncate=False ,vertical=True)

In [ ]:
customer_churn.printSchema()

Display the data type of the coulmns.

In [ ]:
display(customer_churn.summary())

Dropping rows with NaN values

In [ ]:
print("rows: {}".format(customer_churn.count()))
customer_churn = customer_churn.dropna()
print("rows after dropna",format(customer_churn.count()))

Import the pyspark modules required for pre-processing the data.

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [ ]:
output = assembler.transform(customer_churn)

In [ ]:
final_data = output.select('features','churn')

In [ ]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr_churn = LogisticRegression(labelCol='churn')

In [ ]:
fitted_churn_model = lr_churn.fit(train_churn)

In [ ]:
training_sum = fitted_churn_model.summary

In [ ]:
training_sum.predictions.describe().show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [ ]:
pred_and_labels.predictions.show()

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.predictions)
print(auc)

In [ ]:
spark.stop()